# Getting started <img width="223" height="64" src="https://dbatools.io/wp-content/uploads/2016/05/dbatools-logo-1.png" class="custom-logo" alt="dbatools" itemprop="logo">

## Let's se the tables structure

> Note: Confirm that you have `SQL` Kernel selected

In [ ]:
SELECT TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH
  FROM INFORMATION_SCHEMA.COLUMNS

### View `Instances` table

In [ ]:
SELECT *
FROM dbo.Instances

# Let's see how the job looks like

In [ ]:
SELECT j.name, js.step_name, js.subsystem, js.command
FROM msdb.dbo.sysjobsteps AS js
INNER JOIN msdb.dbo.sysjobs AS j
   ON js.job_id = j.job_id
WHERE j.name LIKE '%dbatools%'

<hr>

# How it works?
Create empty dabase from `.bak` template.

> NOTE: Change the Kernel to `PowerShell` before running the following commands

In [ ]:
$restoreSplat = @{
    SqlInstance = "."
    Path = "D:\Presentations\LightUp 2020\Demo\dbatools_demo_201911221208.bak"
    DatabaseName = "dbatools2"
    ReplaceDbNameInFile = $true
    WithReplace = $true
}
Restore-DbaDatabase @restoreSplat 


Confirm that database `dbatools2` exists

In [ ]:
Get-DbaDatabase -SqlInstance . -Database dbatools2

Insert one instance into the table to run the job

In [ ]:
Invoke-DbaQuery -SqlInstance . -Database "dbatools2" -Query "INSERT INTO [dbo].[Instances]([HOSTNAME],[PORT],[ENVIRONMENT],[DOMAIN],[INSTANCE])
VALUES('localhost',1433,'DEVELOPMENT','my.domain.com','MSSQLSERVER')

SELECT * FROM dbo.Instances"

Now, let's invoke the job to collect the data

In [ ]:
$dbatoolsCollectInfo = Get-DbaAgentJob -SqlInstance . -Job "DBATools_CollectInfo"
$dbatoolsCollectInfo.Start()

Confirm that the job is running

In [ ]:
Get-DbaAgentJob -SqlInstance . -Job "DBATools_CollectInfo"

And now we can confirm if we have data on the tables

In [ ]:
Invoke-DbaQuery -SqlInstance . -Database "dbatools2" -Query "SELECT * FROM dbo.[Database]" | Select-Object InstanceName, Name, SizeMB, CollectionTime | Format-Table -AutoSize